## Multi-Agent-System-for-Tourism-and-Travel-Planning

In [55]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
import os

OPENAI_API_KEY = "YOUR API KEY"
SERPER_API_KEY = "YOUR API KEY"

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

Import libraries, APIs and LLM

In [18]:
from crewai import Agent, Crew, Task
import os

In [19]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

Creating Agents

In [20]:
destination_expert = Agent(
    role="Travel Destination Expert",
    goal="Research and recommend optimal travel destinations",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory="Expert travel consultant specializing in matching destinations to traveler preferences"
)

In [21]:
itinerary_planner = Agent(
    role="Itinerary Planner",
    goal="Create efficient and enjoyable travel schedules",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory="Experienced travel coordinator focusing on balanced and practical itineraries"
)

In [22]:
logistics_expert = Agent(
    role="Travel Logistics Specialist",
    goal="Arrange accommodations and transportation",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory="Specialist in finding optimal travel arrangements and accommodations"
)

In [23]:
experience_curator = Agent(
    role="Local Experience Curator",
    goal="Find authentic local activities and experiences",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory="Cultural expert specializing in authentic local experiences and traditions"
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [24]:
from pydantic import BaseModel

# Pydantic models
class DestinationDetails(BaseModel):
    destination: str
    best_time_to_visit: str
    highlights: list
    safety_rating: int
    estimated_budget: float

class AccommodationDetails(BaseModel):
    name: str
    location: str
    type: str
    price_per_night: float
    amenities: list

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [25]:
destination_task = Task(
    description="Find destinations matching {trip_duration}, {budget}, and {interests}",
    expected_output="Destination recommendations with key details",
    human_input=True,
    output_json=DestinationDetails,
    output_file="destination_recommendations.json",
    agent=destination_expert
)

In [26]:
itinerary_task = Task(
    description="Plan daily itinerary for {destination} based on {interests}",
    expected_output="Daily schedule of activities and timing",
    human_input=True,
    output_file="detailed_itinerary.md",
    agent=itinerary_planner
)

In [27]:
logistics_task = Task(
    description="Book accommodations and transport in {destination} for {travelers}",
    expected_output="Travel and accommodation arrangements",
    output_json=AccommodationDetails,
    output_file="travel_logistics.json",
    agent=logistics_expert
)

In [28]:
experience_task = Task(
    description="Find local experiences in {destination} matching {interests}",
    expected_output="Curated local activities and experiences",
    output_file="local_experiences.md",
    agent=experience_curator
)

## Creating the Crew

In [29]:
travel_planning_crew = Crew(
    agents=[destination_expert, itinerary_planner, logistics_expert, experience_curator],
    tasks=[destination_task, itinerary_task, logistics_task, experience_task],
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [30]:
travel_details = {
    'travelers': 2,
    'trip_duration': "10 days",
    'budget': 5000,
    'interests': ["cultural experiences", "food", "photography"],
    'activity_level': "moderate",
    'destination': "Japan",
    'cultural_preferences': ["authentic local cuisine", "traditional ceremonies"],
    'travel_dates': "2025-02-22 to 2025-03-26"
}

In [31]:
result = travel_planning_crew.kickoff(inputs=travel_details)

# Agent: Travel Destination Expert
## Task: Find destinations matching 10 days, 5000, and ['cultural experiences', 'food', 'photography']


# Agent: Travel Destination Expert
## Thought: I need to find destinations that match the traveler's criteria of 10 days duration, a budget of 5000, and interests in cultural experiences, food, and photography.
I should search for destinations that offer a mix of cultural experiences, great food scenes, and picturesque locations within the specified budget and time frame.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"best travel destinations for cultural experiences food photography 10 days under 5000 budget\"}"
## Tool Output: 
{'searchParameters': {'q': 'best travel destinations for cultural experiences food photography 10 days under 5000 budget', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '5 spectacular budget vacation ideas for under $5000', 'link': 'https://www.alliantcreditun

 Nice Work


# Agent: Itinerary Planner
## Task: Plan daily itinerary for Japan based on ['cultural experiences', 'food', 'photography']


# Agent: Itinerary Planner
## Thought: I need to create a daily itinerary for Japan that includes cultural experiences, food, and photography.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Best cultural experiences in Japan\"}"
## Tool Output: 
{'searchParameters': {'q': 'Best cultural experiences in Japan', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Cultural Experiences in Japan | Guide | Travel Japan', 'link': 'https://www.japan.travel/en/guide/cultural-experiences/', 'snippet': "How to experience Japan's rich cultural heritage through traditional activities, from tea ceremonies, taiko drumming and making sushi, to temple stays.", 'position': 1}, {'title': '15x Experience Japanese culture in Japan - Flitter Fever', 'link': 'https://www.flitterfever.com/15x-experience-japanese-culture-in-japan

 looks good


# Agent: Travel Logistics Specialist
## Task: Book accommodations and transport in Japan for 2


# Agent: Travel Logistics Specialist
## Thought: I need to find suitable accommodations and transportation options for a trip to Japan for 2 people. I will start by searching for accommodation options in Japan within the estimated budget.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Best accommodations in Japan within $100 per night\"}"
## Tool Output: 
{'searchParameters': {'q': 'Best accommodations in Japan within $100 per night', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Hotels under $100 USD per night that has onsen or sento? - Reddit', 'link': 'https://www.reddit.com/r/TokyoTravel/comments/1g4ke4o/hotels_under_100_usd_per_night_that_has_onsen_or/', 'snippet': 'Capsule hotels do have shared bathrooms, showers & some onsen/sento under 100 usd per night. Ikebukuro Station & Akihabara Station are major ...', 'position':

In [33]:
import json
from pprint import pprint
from IPython.display import Markdown

In [34]:
with open('destination_recommendations.json') as f:
    pprint(json.load(f))

{'best_time_to_visit': 'Depends on the specific destination chosen',
 'destination': 'Various destinations like Japan, India, Morocco, Peru, Italy, '
                'and Tanzania',
 'estimated_budget': 5000.0,
 'highlights': ['Cultural experiences',
                'Great food scenes',
                'Picturesque locations'],
 'safety_rating': 4}


In [35]:
print("\nDetailed Itinerary:")
Markdown("detailed_itinerary.md")


Detailed Itinerary:


**Day Itinerary for Japan:**

- **Morning:**
  - 8:00 AM - 10:00 AM: Experience a traditional tea ceremony at a tea house in Kyoto.
  
- **Midday:**
  - 12:00 PM - 2:00 PM: Enjoy a delicious sushi-making class in Tokyo.
  
- **Afternoon:**
  - 3:00 PM - 5:00 PM: Visit iconic photography spots such as Fushimi Inari Shrine in Kyoto for photography.
  
- **Evening:**
  - 7:00 PM - 9:00 PM: Indulge in a Kaiseki dining experience in Osaka.

This itinerary offers a balanced mix of cultural experiences, food exploration, and photography opportunities in Japan.

In [36]:

with open('travel_logistics.json') as f:
    pprint(json.load(f))

{'amenities': ['Trains',
               'Buses',
               'Taxis',
               'Ride-hailing services',
               'Narita Express for fast and comfortable travel',
               'Metro and train are the most popular and efficient ways to '
               'travel around Japan, with 1-day passes available for unlimited '
               'travel by metro.',
               'Source: '
               'https://www.jal.co.jp/ar/en/guide-to-japan/plan-your-trip/tips/transportation-options-in-japan.html'],
 'location': 'Japan',
 'name': 'Best transportation options in Japan for travelers',
 'price_per_night': 0.0,
 'type': 'Transportation'}


In [38]:
print("\nLocal Experiences:")
Markdown("local_experiences.md")


Local Experiences:


TOKI offers a wide range of authentic Japanese cultural experiences, from cuisine making to traditional crafts, curated in collaboration with Japan's best artisans. They aim to provide guests with once-in-a-lifetime experiences that showcase the rich cultural heritage of Japan. Testimonials from guests highlight the exceptional quality and uniqueness of the experiences provided by TOKI.

In [39]:
from IPython.display import Markdown
Markdown("local_experiences.md")

TOKI offers a wide range of authentic Japanese cultural experiences, from cuisine making to traditional crafts, curated in collaboration with Japan's best artisans. They aim to provide guests with once-in-a-lifetime experiences that showcase the rich cultural heritage of Japan. Testimonials from guests highlight the exceptional quality and uniqueness of the experiences provided by TOKI.

In [42]:
def generate_travel_report(destination_data, logistics_data, travel_details):
    """Generate a comprehensive markdown travel report"""
    
    report = f"""# {travel_details['destination']} Travel Plan

## Overview
- **Duration:** {travel_details['trip_duration']}
- **Travelers:** {travel_details['travelers']}
- **Dates:** {travel_details['travel_dates']}
- **Budget:** ${travel_details['budget']}

## Destination Insights
- **Best Time to Visit:** {destination_data['best_time_to_visit']}
- **Safety Rating:** {destination_data['safety_rating']}/10
- **Budget Estimate:** ${destination_data['estimated_budget']}

### Key Highlights
{chr(10).join('- ' + highlight for highlight in destination_data['highlights'])}

## Accommodation
**{logistics_data['name']}**
- Location: {logistics_data['location']}
- Type: {logistics_data['type']}
- Price: ${logistics_data['price_per_night']}/night
- Amenities: {', '.join(logistics_data['amenities'])}

## Daily Schedule
"""
    
    # Add itinerary if available
    try:
        with open('detailed_itinerary.md', 'r') as f:
            report += f"\n{f.read()}\n"
    except FileNotFoundError:
        report += "\n*Itinerary being processed...*\n"

    # Add experiences if available
    report += "\n## Local Experiences\n"
    try:
        with open('local_experiences.md', 'r') as f:
            report += f"\n{f.read()}\n"
    except FileNotFoundError:
        report += "\n*Experiences being curated...*\n"

    report += """
## Travel Essentials
1. **Documentation**
   - Passport/Visa
   - Travel Insurance
   - Booking Confirmations

2. **Health & Safety**
   - Emergency Contacts
   - Local Embassy
   - Travel Insurance Details

3. **Packing List**
   - Weather-appropriate Clothing
   - Travel Adapters
   - Essential Documents
"""

    return report

In [43]:
from IPython.display import Markdown

# After your crew execution, generate and display the report
result = travel_planning_crew.kickoff(inputs=travel_details)

# Load data files
with open('destination_recommendations.json') as f:
    destination_data = json.load(f)
with open('travel_logistics.json') as f:
    logistics_data = json.load(f)

# Generate and display report
report = generate_travel_report(destination_data, logistics_data, travel_details)
display(Markdown(report))

# Save report
with open('travel_report.md', 'w') as f:
    f.write(report)

# Agent: Travel Destination Expert
## Task: Find destinations matching 10 days, 5000, and ['cultural experiences', 'food', 'photography']


# Agent: Travel Destination Expert
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"best travel destinations for cultural experiences food photography 10 days under 5000\"}"
## Tool Output: 
{'searchParameters': {'q': 'best travel destinations for cultural experiences food photography 10 days under 5000', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '5 spectacular budget vacation ideas for under $5000', 'link': 'https://www.alliantcreditunion.org/money-mentor/5-spectacular-budget-vacation-ideas-for-under-5000', 'snippet': "It's hard to miss when it comes to Europe – cafes, museums, architecture, culture. The continent's rich and diverse history make it a wonderful destination for ...", 'position': 1}, {'title': '25 Spectacular Travel Photography Destinations (With Photos)', 'link': 'htt

 looks good


# Agent: Itinerary Planner
## Task: Plan daily itinerary for Japan based on ['cultural experiences', 'food', 'photography']


# Agent: Itinerary Planner
## Thought: I need to gather information on cultural experiences, food, and photography activities in Japan to create a daily itinerary.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"top cultural experiences in Japan\"}"
## Tool Output: 
{'searchParameters': {'q': 'top cultural experiences in Japan', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Cultural Experiences in Japan | Guide | Travel Japan', 'link': 'https://www.japan.travel/en/guide/cultural-experiences/', 'snippet': "How to experience Japan's rich cultural heritage through traditional activities, from tea ceremonies, taiko drumming and making sushi, to temple stays.", 'position': 1}, {'title': '15x Experience Japanese culture in Japan - Flitter Fever', 'link': 'https://www.flitterfever.com/15x-experience-japane

 looks good


# Agent: Travel Logistics Specialist
## Task: Book accommodations and transport in Japan for 2


# Agent: Travel Logistics Specialist
## Thought: I need to find suitable accommodations and transportation options in Japan for 2 people based on the provided itinerary.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Best accommodations in Japan for traditional stay\"}"
## Tool Output: 
{'searchParameters': {'q': 'Best accommodations in Japan for traditional stay', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'THE 10 BEST Japan Ryokan 2025 (with Prices) - Tripadvisor', 'link': 'https://www.tripadvisor.com/Hotels-g294232-zff9-Japan-Hotels.html', 'snippet': "A Ryokan is a traditional Japanese accommodation which typically features 'futon' (folding mattresses) on 'tatami' (straw mat) floors.", 'position': 1}, {'title': 'The 15 Best Ryokans To Stay In When Visiting Japan - Culture Trip', 'link': 'https://theculturetrip.com/asia/ja

# Japan Travel Plan

## Overview
- **Duration:** 10 days
- **Travelers:** 2
- **Dates:** 2025-02-22 to 2025-03-26
- **Budget:** $5000

## Destination Insights
- **Best Time to Visit:** Spring or Fall
- **Safety Rating:** 8/10
- **Budget Estimate:** $2500.0

### Key Highlights
- Exploring the souks and medinas of Marrakech and Fes
- Capturing the colorful streets and historic buildings in Chefchaouen
- Photographing the dramatic landscapes of the Atlas Mountains and Sahara Desert
- Meeting friendly locals for portrait photography opportunities

## Accommodation
**Gora Kadan**
- Location: Hakone, Japan
- Type: Ryokan
- Price: $450.0/night
- Amenities: Luxurious baths, Traditional Japanese cuisine, Indoor-outdoor spaces

## Daily Schedule

Day 1:
- Morning: Participate in a Japanese tea ceremony to immerse in the rich cultural heritage.
- Afternoon: Enjoy a traditional stay at a ryokan for a deeper insight into Japanese culture.
- Evening: Explore the streets and capture the beauty of geisha in select areas.

Day 2:
- Morning: Join a photography tour to capture stunning landscapes and cultural sites.
- Afternoon: Experience Japanese cuisine through a sushi-making class.
- Evening: Attend a local festival (matsuri) for unique photo opportunities.

Day 3:
- Morning: Take part in a flower arrangement (ikebana) workshop for a different cultural perspective.
- Afternoon: Explore the city and capture the vibrant street life and architecture.
- Evening: Relax and enjoy a traditional Japanese meal at an izakaya or fine dining restaurant.

Day 4:
- Full Day Photography Tour: Visit iconic photography spots like Mount Fuji, temples, and gardens.
- Evening: Attend a sumo match for a glimpse into Japan's national sport.

Day 5:
- Morning: Learn more about Japanese history and culture through a temple stay (shukubo experience).
- Afternoon: Explore local markets and capture the essence of daily life.
- Evening: Participate in a taiko drum performance for an immersive cultural experience.

Thought: I now know the final answer

## Local Experiences

Asakusa cultural tour with guide about the history and culture, making traditional Japanese food samples, participating in photography and food tour in Setagaya, and exploring culinary adventures curated by Arigato Travel for authentic local experiences in Japan related to cultural experiences, food, and photography.
```

## Travel Essentials
1. **Documentation**
   - Passport/Visa
   - Travel Insurance
   - Booking Confirmations

2. **Health & Safety**
   - Emergency Contacts
   - Local Embassy
   - Travel Insurance Details

3. **Packing List**
   - Weather-appropriate Clothing
   - Travel Adapters
   - Essential Documents
